<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Коннект-к-БД" data-toc-modified-id="Коннект-к-БД-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Коннект к БД</a></span></li><li><span><a href="#Выгрузка-данных-из-БД" data-toc-modified-id="Выгрузка-данных-из-БД-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Выгрузка данных из БД</a></span></li><li><span><a href="#Построение-дашборда-в-Tableau-Public" data-toc-modified-id="Построение-дашборда-в-Tableau-Public-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Построение дашборда в Tableau Public</a></span></li><li><span><a href="#Создание-презентации-для-менеджеров" data-toc-modified-id="Создание-презентации-для-менеджеров-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Создание презентации для менеджеров</a></span></li></ul></div>

# Построение дашборда

При работе в сервисе отображения статей производился анализ пользовательского взаимодействия с карточками статей.  
Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».
Пользователей системы характеризует возрастная категория. Например, «26-30» или «45+».  
Есть три способа взаимодействия пользователей с системой:
- карточка отображена для пользователя (show);
- пользователь кликнул на карточку (click);
- пользователь просмотрел статью карточки (view).  

По запросу менеджеров было принято решение сделать дашборд для быстрого анализа необходимых данных и ответов на следующие вопросы:
- Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много карточек генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?  

Принято решение, что дашборд должен основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу.  
С менеджерами было проведено обсуждение состава дашборда, его внешний вид и набор отображаемых данных. После этого проведено обсуждение с администраторами БД с целью выяснения куда и как собираются нужные данные. С ними же решено, где хранить агрегирующие таблицы. Совместно был разработан пайплайн сбора и агрегации данных и схема дашборда.  
Было сформировано следующее техническое задание:
1. Бизнес-задача: анализ взаимодействия пользователей с карточками;
2. Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
3. Кто будет основным пользователем дашборда: менеджеры по анализу контента;
4. Состав данных для дашборда:
    - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    - Разбивка событий по темам источников;
    - Таблица соответствия тем источников темам карточек;
5. По каким параметрам данные должны группироваться:
    - Дата и время;
    - Тема карточки;
    - Тема источника;
    - Возрастная группа;
6. Характер данных:
    - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    - Разбивка событий по темам источников — относительные величины (% событий);
    - Соответствия тем источников темам карточек - абсолютные величины;
7. Важность: все графики имеют равную важность;
8. Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица `log_raw`);
9. База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД `zen`;
10. Частота обновления данных: один раз в сутки, в полночь по UTC;
11. Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):
![Макет дашборда](https://i.ibb.co/d7Sz9Qr/dash-maket.png)

**Цель работы:** Построить дашборд для менеджеров, отвечающий на поставленные вопросы. Также оформить презентацию с ответами.

**Ход работы:**
1. Создать коннект к БД.
2. Выгрузить данные из БД в .csv-файл.
3. В Tableau Public построить дашборд в соответствии с ТЗ.
4. Сделать презентацию для менеджеров.


## Коннект к БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Задаём параметры подключения к БД, которые указал админ.
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных
# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
# Подключаемся к БД.
engine = create_engine(connection_string)

## Выгрузка данных из БД

In [3]:
# Формируем sql-запрос на выгрузку сырых данных.
query = ''' SELECT *
            FROM log_raw
        '''
# Выполняем запрос и сохраняем результат выполнения в DataFrame.
log_raw = pd.io.sql.read_sql(query, con=engine)
# Смотрим, что выгрузили
log_raw.head()

,event_id,age_segment,event,item_id,item_topic,item_type,source_id,source_topic,source_type,ts,user_id
0,2931262,18-25,show,1793260,Красота,native,8388799,История,publisher,1569349657638,2185026
1,2931277,18-25,show,7465641,Культура,native,5837959,История,publisher,1569349657971,6845238
2,2931285,18-25,show,406073,История,url,2335872,Сад и дача,domain,1569349658408,5244363
3,2931247,26-30,show,752665,Путешествия,native,6963489,Здоровье,publisher,1569349657189,7553074
4,2931278,26-30,click,6843081,Юмор,url,5427748,Семейные отношения,domain,1569349658054,1317585


После выгрузки сырых данных необходимо провести агрегацию.  
Для построения дашборда по ТЗ нам необходима следующая информация из сырых данных:
- тема карточки `item_topic`;
- тема источника `source_topic`;
- возрастная категория `age_segment`;
- время `ts`;
- количество взаимодействий с карточкой.  

Приведем время в понятный формат, округлив до минут. Затем сгруппируем таблицу по столбцам тем, возраста и времени, подсчитывая количество взаимодействий.


In [4]:
# Преобразуем данные к нужным типам.
log_raw['dt'] = pd.to_datetime(log_raw['ts'], unit='ms').dt.round('T')

# Готовим агрегирующую таблицу.
data_dash = (
    log_raw
    .groupby(['item_topic', 'source_topic', 'age_segment', 'dt'])
    .agg({'event': 'count'})
    .reset_index()
)
data_dash.head()

,item_topic,source_topic,age_segment,dt,event
0,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,Деньги,Авто,18-25,2019-09-24 18:56:00,27


База данных в данное время доступна только в режиме read-only. Для записи агрегирующих таблиц в базу данных мог использоваться код ниже.

In [5]:
# Сохраняем датафрейм в таблицу data_dash базы данных.
# data_dash.to_sql(
#     name='data_dash', con=engine, if_exists='append', index=False
# ) 

Также возможно написание скрипта для еженедельной выгрузки данных по данному пайплайну. В данном проекте это не рассматривалось, т.к. данный скрипт выполнить невозможно из-за необновляемости БД и отсутсвия доступа на запись. Однако, заготовку данного скрипта можно найти в этой же папке на гитхабе, что и эта тетрадь.

Аггрегированную таблицу из сырых выгруженных данных сохраним локально для работы с ней далее в Tableau Public.

In [6]:
data_dash.to_csv('data_dash.csv', index=False) 

Теперь можно переходить к построению дашборда в Tableau Public.

## Построение дашборда в Tableau Public

По согласованному ТЗ был создан дашборд, который опубликован в Tableau Public:  
https://public.tableau.com/views/Card_Dash_for_studies/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

## Создание презентации для менеджеров

Для менеджеров была создана презентация, которую можно посмотреть здесь:  
https://drive.google.com/file/d/1E3X2OxoehqNbIVIjuxDHuMOb3HAI5Dcj/view?usp=sharing